<a href="https://colab.research.google.com/github/medaminechafii/kaggle-competitions/blob/main/kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1612]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [1613]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [1614]:
print("shape of training set : ",train.shape)
train.head()

shape of training set :  (2190, 13)


,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


In [1615]:
print("shape of testing set : ",test.shape)
test.head()

shape of testing set :  (730, 12)


,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
1,2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2,2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
3,2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
4,2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4


In [1616]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             730 non-null    int64  
 1   day            730 non-null    int64  
 2   pressure       730 non-null    float64
 3   maxtemp        730 non-null    float64
 4   temparature    730 non-null    float64
 5   mintemp        730 non-null    float64
 6   dewpoint       730 non-null    float64
 7   humidity       730 non-null    float64
 8   cloud          730 non-null    float64
 9   sunshine       730 non-null    float64
 10  winddirection  729 non-null    float64
 11  windspeed      730 non-null    float64
dtypes: float64(10), int64(2)
memory usage: 68.6 KB


##Feature engineering

In [1617]:
extra_data = pd.read_csv('Rainfall.csv')
days = [i+1 for i in range(len(extra_data))]
extra_data["day"] = np.array(days)+2189

In [1618]:
print(train["id"].max())

2189


In [1619]:
extra_data["id"] = np.array(days) + 2189

In [1620]:
extra_data["rainfall"] = extra_data["rainfall"].map({"yes":1,"no":0})

In [1621]:
extra_data.head(10)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed,id
0,2190,1025.9,19.9,18.3,16.8,13.1,72,49,1,9.3,80.0,26.3,2190
1,2191,1022.0,21.7,18.9,17.2,15.6,81,83,1,0.6,50.0,15.3,2191
2,2192,1019.7,20.3,19.3,18.0,18.4,95,91,1,0.0,40.0,14.2,2192
3,2193,1018.9,22.3,20.6,19.1,18.8,90,88,1,1.0,50.0,16.9,2193
4,2194,1015.9,21.3,20.7,20.2,19.9,95,81,1,0.0,40.0,13.7,2194
5,2195,1018.8,24.3,20.9,19.2,18.0,84,51,1,7.7,20.0,14.5,2195
6,2196,1021.8,21.4,18.8,17.0,15.0,79,56,0,3.4,30.0,21.5,2196
7,2197,1020.8,21.0,18.4,16.5,14.4,78,28,0,7.7,60.0,14.3,2197
8,2198,1020.6,18.9,18.1,17.1,14.3,78,79,0,3.3,70.0,39.3,2198
9,2199,1017.5,18.5,18.0,17.2,15.5,85,91,1,0.0,70.0,37.7,2199


In [1622]:
extra_data.columns = ["day","pressure","maxtemp","temparature","mintemp","dewpoint","humidity","cloud","rainfall","sunshine","winddirection","windspeed","id"]

In [1623]:
m = extra_data["winddirection"].median()
m1 = extra_data["windspeed"].median()
extra_data["winddirection"].fillna(m,inplace = True)
extra_data["windspeed"].fillna(m1,inplace = True)
test["winddirection"].fillna(m,inplace = True)

In [1624]:
full_data = pd.concat([train,extra_data],ignore_index = True,axis = 0).copy()

In [1625]:
full_data["temp_diff"] = full_data["maxtemp"] - full_data["mintemp"]
test["temp_diff"] = test["maxtemp"] - test["mintemp"]

In [1626]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 4)
df = km.fit_predict(pd.DataFrame(full_data).loc[:,["winddirection","temparature"]])
df_test = km.fit_predict(pd.DataFrame(test).loc[:,["winddirection","temparature"]])

In [1627]:
full_data.insert(3,"winddir/temp_km",df)
test.insert(3,"winddir/temp_km",df_test)

In [1628]:
full_data.head()

,id,day,pressure,winddir/temp_km,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,temp_diff
0,0,1,1017.4,0,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1,1.3
1,1,2,1019.5,0,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1,0.4
2,2,3,1024.1,2,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1,4.8
3,3,4,1013.4,0,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1,1.2
4,4,5,1021.8,0,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0,6.1


In [1629]:
from sklearn.feature_selection import mutual_info_regression

In [1630]:
full_data["cloud_sun"] = (full_data["cloud"]*full_data["sunshine"])/10
test["cloud_sun"] = (test["cloud"]*test["sunshine"])/10

In [1631]:
full_data["sunshine_humidity"] = (full_data["humidity"]*full_data["sunshine"])/10
test["sunshine_humidity"] = (test["humidity"]*test["sunshine"])/10

In [1632]:
km2 = KMeans(n_clusters = 5)
df2 = km2.fit_predict(pd.DataFrame(full_data).loc[:,["cloud","sunshine"]])
df2_test = km2.fit_predict(pd.DataFrame(test).loc[:,["cloud","sunshine"]])
full_data.insert(3,"could/sunshine_km",df2)
test.insert(3,"could/sunshine_km",df2_test)

In [1633]:
km3 = KMeans(n_clusters=2)
df3 = km2.fit_predict(pd.DataFrame(full_data).loc[:,["sunshine","temp_diff"]])
df3_test = km2.fit_predict(pd.DataFrame(test).loc[:,["sunshine","temp_diff"]])
test.insert(3,"sunshine/maxmintemp_km",df3_test)
full_data.insert(3,"sunshine/maxmintemp_km",df3)

In [1634]:
to_oh = ["winddir/temp_km","could/sunshine_km","sunshine/maxmintemp_km"]

In [1635]:
RMV = ['rainfall','id',"day"]+to_oh
FEATURES = [c for c in full_data.columns if not c in RMV]
print("Our features are:")
print( FEATURES )

Our features are:
['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed', 'temp_diff', 'cloud_sun', 'sunshine_humidity']


In [1636]:
N_Folds = 5

In [1637]:
n = len(FEATURES)

In [1638]:
full_data["day"] = full_data["day"]%30

In [1639]:
WGT = WGT = {'day':1.2/n, 'pressure': 1/n, 'maxtemp': 1/n, 'temparature': 1/n, 'mintemp': 1/n, 'dewpoint': 1/n, 'humidity': 1.8/n,
       'cloud': 4/n, 'sunshine': 2/n, 'winddirection': 1/n, 'windspeed': 1/n,"temp_diff":1/n,"cloud_sun":3.4/n,"sunshine_humidity":2.7/n}

In [1640]:
from sklearn.preprocessing import OneHotEncoder

In [1641]:
OH = OneHotEncoder(handle_unknown = "ignore",sparse_output = False)
OH_data = pd.DataFrame(OH.fit_transform(full_data[to_oh]))
OH_test = pd.DataFrame(OH.transform(test[to_oh]))
OH_data.index = full_data.index
OH_test.index = test.index
OH_data.columns = OH.get_feature_names_out()
OH_test.columns = OH.get_feature_names_out()
full_data = full_data.drop(to_oh,axis = 1)
test = test.drop(to_oh,axis = 1)
full_data = pd.concat([full_data,OH_data],axis = 1)
test = pd.concat([test,OH_test],axis = 1)

In [1642]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

##FULL

In [1643]:
def knn_model(train,test,FOLDS,FEATURES):
  oof_knn = np.zeros(len(train))
  pred_knn = np.zeros(len(test))
  kf = KFold(n_splits=FOLDS, shuffle=True)
  for i,(train_index, test_index) in enumerate(kf.split(train)):
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"rainfall"]
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"rainfall"]
    x_test = test[FEATURES].copy()
    for c in FEATURES:
      m = x_train[c].median()
      s = x_train[c].std()
      x_train[c] = WGT[c]*(x_train[c]-m)/s
      x_valid[c] = WGT[c]*(x_valid[c]-m)/s
      x_test[c] = WGT[c]*(x_test[c]-m)/s
      x_test[c] = x_test[c].fillna(m)
      x_train[c] = x_train[c].fillna(m)
    knn = KNeighborsClassifier(n_neighbors = 100,p = 1)
    knn.fit(x_train.values,y_train.values)
    oof_knn[test_index] = knn.predict_proba(x_valid.values)[:,1]
    pred_knn += knn.predict_proba(x_test.values)[:,1]
    pred_knn /= FOLDS
  return oof_knn,pred_knn

In [1644]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [1645]:
def lr_model(train,test,FOLDS,FEATURES):
  oof_lr = np.zeros(len(train))
  pred_lr = np.zeros(len(test))
  kf = KFold(n_splits=FOLDS, shuffle=True)
  for i,(train_index, test_index) in enumerate(kf.split(train)):
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"rainfall"]
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"rainfall"]
    x_test = test[FEATURES].copy()
    for c in FEATURES:
      m = x_train[c].median()
      s = x_train[c].std()
      x_train[c] = WGT[c]*(x_train[c]-m)/s
      x_valid[c] = WGT[c]*(x_valid[c]-m)/s
      x_test[c] = WGT[c]*(x_test[c]-m)/s
      x_test[c] = x_test[c].fillna(m)
      x_train[c] = x_train[c].fillna(m)
    lr = LogisticRegression(max_iter=100)
    lr.fit(x_train.values,y_train.values)
    oof_lr[test_index] = lr.predict_proba(x_valid.values)[:,1]
    pred_lr += lr.predict_proba(x_test.values)[:,1]
    pred_lr /= FOLDS
  return oof_lr,pred_lr

In [1646]:
def rf_model (train,test,FOLDS,FEATURES):
  oof_rf = np.zeros(len(train))
  pred_rf = np.zeros(len(test))
  kf = KFold(n_splits = FOLDS, shuffle = True)
  for i,(train_index, test_index) in enumerate(kf.split(train)):
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"rainfall"]
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"rainfall"]
    x_test = test[FEATURES].copy()
    for c in FEATURES:
      m = x_train[c].median()
      s = x_train[c].std()
      x_train[c] = WGT[c]*(x_train[c]-m)/s
      x_valid[c] = WGT[c]*(x_valid[c]-m)/s
      x_test[c] = WGT[c]*(x_test[c]-m)/s
      x_test[c] = x_test[c].fillna(m)
      x_train[c] = x_train[c].fillna(m)
    rf = RandomForestClassifier(n_estimators = 100)
    rf.fit(x_train.values,y_train.values)
    oof_rf[test_index] = rf.predict_proba(x_valid.values)[:,1]
    pred_rf += rf.predict_proba(x_test.values)[:,1]
    pred_rf /= FOLDS
  return oof_rf,pred_rf

In [1647]:
import xgboost as xgb
from xgboost import XGBClassifier,XGBRegressor

In [1648]:
def xgb_model (train,test,FOLDS,FEATURES):
  oof_xgb = np.zeros(len(train))
  pred_xgb = np.zeros(len(test))
  kf = KFold(n_splits = FOLDS, shuffle = True)
  for i,(train_index, test_index) in enumerate(kf.split(train)):
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"rainfall"]
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"rainfall"]
    x_test = test[FEATURES].copy()

  model = XGBClassifier(
      device="cuda",
      max_depth=6,
      colsample_bytree=0.9,
      subsample=0.9,
      n_estimators=1000,
      learning_rate=0.1,
      eval_metric="auc",
      early_stopping_rounds=100,
      alpha=1,
    )
  model.fit(
      x_train, y_train,
      eval_set=[(x_valid, y_valid)],verbose = 100
    )
  oof_xgb[test_index] = model.predict_proba(x_valid.values)[:,1]
  pred_xgb += model.predict_proba(x_test.values)[:,1]
  pred_xgb /= FOLDS

  return oof_xgb,pred_xgb

In [1649]:
from sklearn.metrics import roc_auc_score

In [1650]:
from scipy.optimize import minimize

In [1651]:
def find_optimal_weights(oof_preds,true_vals):
  initial_weights = np.array([0.1,0.1,0.1])
  bounds = [(0,1) for i in range(len(oof_preds))]
  def constraint(weights):
    return weights.sum()-1
  constraints = {"type":"eq","fun":constraint}
  def objective(weights):
    weighted_preds = np.zeros(len(true_vals),dtype = float)
    for i,pred in enumerate(oof_preds):
      weighted_preds += weights[i]*pred
    return -roc_auc_score(true_vals,weighted_preds)
  result = minimize(objective,initial_weights,method = "SLSQP",constraints = constraints,bounds = bounds)
  weights = result["x"]
  print(f"Optimization successful: {result['success']}")
  print(f"Optimal weights: {weights}")
  ensemble_pred = np.zeros(len(true_vals),dtype = float)
  for i,pred in enumerate(oof_preds):
    ensemble_pred += weights[i]*pred
  ensemble_auc = roc_auc_score(true_vals, ensemble_pred)
  print(f"Ensemble AUC: {ensemble_auc:.5f}")
  return weights

In [1652]:
def run_ensemble(train,test,FEATURES):
  print("Running KNN model...")
  oof_knn, pred_knn = knn_model(train, test, N_Folds,FEATURES)

  print("Running logistic regression model...")
  oof_logistic, pred_logistic = lr_model(train, test, N_Folds,FEATURES)

  print("Running random forest model...")
  oof_rf, pred_rf = rf_model(train, test, N_Folds,FEATURES)

    # Calculate individual model AUC scores
  true_values = train["rainfall"].values
  knn_auc = roc_auc_score(true_values, oof_knn)
  logistic_auc = roc_auc_score(true_values, oof_logistic)
  rf_auc = roc_auc_score(true_values, oof_rf)

  print(f"KNN AUC: {knn_auc:.5f}")
  print(f"Logistic Regression AUC: {logistic_auc:.5f}")
  print(f"Random Forest AUC: {rf_auc:.5f}")

    # Find optimal weights using OOF predictions
  print("\nFinding optimal ensemble weights...")
  oof_preds = [oof_knn, oof_logistic, oof_rf]
  optimal_weights = find_optimal_weights(oof_preds, true_values)

    # Apply weights to test predictions
  print("\nApplying weights to test predictions...")
  test_preds = [pred_knn, pred_logistic, pred_rf]
  ensemble_test_preds = np.zeros(len(test))

  for i, pred in enumerate(test_preds):
      ensemble_test_preds += optimal_weights[i] * pred

  print("Ensemble test predictions ready!")

    # Return both the ensemble predictions and the individual model predictions
  return {
        "ensemble": ensemble_test_preds,
        "knn": pred_knn,
        "logistic": pred_logistic,
        "random_forest": pred_rf
    }

In [1653]:
my_dictionary = run_ensemble(full_data,test,FEATURES)

Running KNN model...
Running logistic regression model...
Running random forest model...
KNN AUC: 0.88334
Logistic Regression AUC: 0.89134
Random Forest AUC: 0.88148

Finding optimal ensemble weights...
Optimization successful: True
Optimal weights: [0.33333333 0.33333333 0.33333333]
Ensemble AUC: 0.89040

Applying weights to test predictions...
Ensemble test predictions ready!


In [1659]:
ensemble_preds = pd.DataFrame(my_dictionary["ensemble"])
knn_preds = pd.DataFrame(my_dictionary["knn"])
logistic_preds = pd.DataFrame(my_dictionary["logistic"])
rf_preds = pd.DataFrame(my_dictionary["random_forest"])

In [1660]:
index = test.id

In [1661]:
#pd.concat([index,ensemble_preds],axis = 1).set_index("id").to_csv("ensemble_preds1.csv")
#pd.concat([index,knn_preds],axis = 1).set_index("id").to_csv("knn_preds1.csv")
#pd.concat([index,logistic_preds],axis = 1).set_index("id").to_csv("logistic_preds1.csv")
#pd.concat([index,rf_preds],axis = 1).set_index("id").to_csv("rf_preds1.csv")